In [14]:
"""
Week2 机器学习综合练习

本练习涵盖以下内容：
1. 数据处理（数据清洗、特征工程）
2. 分类任务（判断是否为篮球运动员）
3. 回归任务（房价预测）
4. 模型评估（分类和回归的评估指标）

请按照要求完成以下任务。
"""

import pandas as pd
import numpy as np
from pandas.core.generic import clean_fill_method
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    mean_absolute_error, mean_squared_error, r2_score
)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# ============================================================================
# 任务1：数据处理 - 数据清洗和特征工程
# ============================================================================

def task1_data_processing():
    """
    任务1：数据处理

    给定一个包含"脏"数据的DataFrame，请完成以下操作：
    1. 处理缺失值（用均值填充数值型特征，用众数填充类别型特征）
    2. 处理异常值（年龄应该在18-100之间，房价应该在0-10000之间）
    3. 对类别型特征进行编码（使用LabelEncoder）
    4. 对数值型特征进行标准化（使用StandardScaler）

    返回处理后的DataFrame
    """
    # 创建包含"脏"数据的示例数据
    data = {
        'Name': ['Mike', 'Jerry', 'Bryan', 'Patricia', 'Elodie', 'Remy', 'John', 'Marine', 'Julien', 'Fred'],
        'City': ['Miami', 'New York', 'Orlando', 'Miami', 'Phoenix', 'Chicago', 'New York', 'Miami', None, 'Orlando'],
        'Age': [42, 32, 18, 45, 35, 72, 48, 45, 52, 200],  # 包含异常值200
        'Salary': [5000, 6000, None, 5500, 5800, 7000, 6500, 5200, 6800, 3000],
        'HousePrice': [1000, 1300, 700, 1100, 850, 1500, 1200, 1050, 1400, -100]  # 包含异常值-100
    }
    df = pd.DataFrame(data)

    print("=" * 60)
    print("任务1：数据处理")
    print("=" * 60)
    print("\n原始数据：")
    print(df)
    print("\n数据信息：")
    print(df.info())
    print("\n缺失值统计：")
    print(df.isnull().sum())

    # TODO: 请在此处完成数据处理
    # 提示：
    # 1. 处理缺失值
    # 2. 处理异常值（Age应该在18-100之间，HousePrice应该在0-10000之间）
    # 3. 对City进行编码
    # 4. 对数值型特征进行标准化

    # 你的代码写在这里
    df_processed = df.copy()  # 请修改这里

    # 1. 处理缺失值
    # 对于数值型列，使用均值填充
    numeric_cols = df_processed.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if df_processed[col].isnull().sum() > 0:
            mean_value = df_processed[col].mean()
            df_processed[col].fillna(mean_value, inplace=True)
            print(f"\n列 '{col}': 使用均值 {mean_value:.2f} 填充缺失值")

    # 对于类别型列，使用众数填充
    categorical_cols = df_processed.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        if df_processed[col].isnull().sum() > 0:
            mode_value = df_processed[col].mode()[0] if len(df_processed[col].mode()) > 0 else '未知'
            df_processed[col].fillna(mode_value, inplace=True)
            print(f"列 '{col}': 使用众数 '{mode_value}' 填充缺失值")

    # 2. 处理异常值
    def detect_outliers_iqr(df, column):
        """使用IQR方法检测异常值"""
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
        return outliers, lower_bound, upper_bound

    def detect_outliers_zscore(df, column, threshold=3):
        """使用Z-score方法检测异常值"""
        z_scores = np.abs((df[column] - df[column].mean()) / df[column].std())
        outliers = df[z_scores > threshold]
        return outliers

    print("\n开始异常值检测和处理...")

    # 对数值型列进行异常值检测和处理
    outlier_summary = []

    for col in numeric_cols:
        print(f"\n--- 分析列: {col} ---")

        # IQR方法检测
        outliers_iqr, lower_bound, upper_bound = detect_outliers_iqr(df_processed, col)
        outlier_count_iqr = len(outliers_iqr)

        # Z-score方法检测
        outliers_zscore = detect_outliers_zscore(df_processed, col)
        outlier_count_zscore = len(outliers_zscore)

        print(f"  IQR方法检测到异常值: {outlier_count_iqr} 个")
        print(f"  正常值范围: [{lower_bound:.2f}, {upper_bound:.2f}]")
        print(f"  Z-score方法检测到异常值: {outlier_count_zscore} 个")

        if outlier_count_iqr > 0:
            print(f"  异常值示例:")
            print(outliers_iqr[[col]].head(3))

            # 使用边界值替换异常值
            df_processed.loc[df_processed[col] < lower_bound, col] = lower_bound
            df_processed.loc[df_processed[col] > upper_bound, col] = upper_bound
            print(f"  已使用IQR边界值替换异常值")

        # 应用业务规则限制
        if col == 'Age':
            # 年龄应该在18-100之间
            before_count = len(df_processed[(df_processed[col] < 18) | (df_processed[col] > 100)])
            df_processed[col] = df_processed[col].clip(18, 100)
            after_count = len(df_processed[(df_processed[col] < 18) | (df_processed[col] > 100)])
            print(f"  应用业务规则: Age限制在18-100之间，修正了 {before_count - after_count} 个值")
        elif col == 'HousePrice':
            # 房价应该在0-10000之间
            before_count = len(df_processed[(df_processed[col] < 0) | (df_processed[col] > 10000)])
            df_processed[col] = df_processed[col].clip(0, 10000)
            after_count = len(df_processed[(df_processed[col] < 0) | (df_processed[col] > 10000)])
            print(f"  应用业务规则: HousePrice限制在0-10000之间，修正了 {before_count - after_count} 个值")

        outlier_summary.append({
            '列名': col,
            'IQR异常值数量': outlier_count_iqr,
            'IQR下界': lower_bound,
            'IQR上界': upper_bound,
            'Z-score异常值数量': outlier_count_zscore
        })

    # 显示异常值处理总结
    if outlier_summary:
        print("\n异常值处理总结:")
        summary_df = pd.DataFrame(outlier_summary)
        print(summary_df)

    print(f'\n处理完异常值之后的数据：{df_processed}\n')

    # 3. 对类别型特征进行编码（使用LabelEncoder）
    # 对 City 列进行编码
    le_city = LabelEncoder()
    df_processed['City_encoded'] = le_city.fit_transform(df_processed['City'])
    print(f"\n城市编码映射：{dict(zip(le_city.classes_, le_city.transform(le_city.classes_)))}")

    # 4. 对数值型特征进行标准化（使用StandardScaler）
    scaler = StandardScaler()
    df_processed[['Age', 'Salary', 'HousePrice']] = scaler.fit_transform(df_processed[['Age', 'Salary', 'HousePrice']])
    print(f"\n数值特征标准化完成：['Age', 'Salary', 'HousePrice']")





    print("\n处理后的数据：")
    print(df_processed)
    print("\n处理后的数据信息：")
    print(df_processed.info())

    return df_processed

In [ ]:
# ============================================================================
# 任务2：分类任务 - 判断是否为篮球运动员
# ============================================================================

def task2_classification():
    """
    任务2：分类任务

    使用处理后的数据训练一个分类模型，判断一个人是否为篮球运动员。

    要求：
    1. 准备训练数据（使用任务1处理后的数据）
    2. 分割训练集和测试集（比例8:2）
    3. 训练一个分类模型（可以使用LogisticRegression或DecisionTreeClassifier）
    4. 在测试集上评估模型性能
    5. 输出准确率、混淆矩阵和分类报告

    返回训练好的模型和评估结果
    """
    print("\n" + "=" * 60)
    print("任务2：分类任务 - 判断是否为篮球运动员")
    print("=" * 60)

    # 创建训练数据（基于演讲稿中的例子）
    train_data = {
        'City': ['Miami', 'New York', 'Orlando', 'Miami', 'Phoenix', 'Chicago', 'New York'],
        'Age': [42, 32, 18, 45, 35, 72, 48],
        'IsBasketballPlayer': [1, 0, 0, 1, 0, 1, 1]  # 1表示是，0表示否
    }
    df_train = pd.DataFrame(train_data)

    # 创建测试数据
    test_data = {
        'City': ['Miami', 'Miami', 'Orlando', 'Boston', 'Phoenix'],
        'Age': [45, 52, 20, 34, 90],
        'IsBasketballPlayer': [1, 1, 0, 0, 0]  # 真实标签（用于评估）
    }
    df_test = pd.DataFrame(test_data)

    print("\n训练数据：")
    print(df_train)
    print("\n测试数据：")
    print(df_test)

    # TODO: 请在此处完成分类任务
    # 提示：
    # 1. 对City进行编码
    # 2. 准备特征X和标签y
    # 3. 分割数据（如果需要）
    # 4. 训练模型
    # 5. 预测并评估

    # 你的代码写在这里
    model = None  # 请修改这里
    accuracy = 0  # 请修改这里
    cm = None  # 请修改这里

    print("\n模型准确率：", accuracy)
    print("\n混淆矩阵：")
    print(cm)
    print("\n分类报告：")
    # 请添加分类报告的打印

    return model, accuracy, cm

In [ ]:
# ============================================================================
# 任务3：回归任务 - 房价预测
# ============================================================================

def task3_regression():
    """
    任务3：回归任务

    使用处理后的数据训练一个回归模型，预测房价。

    要求：
    1. 准备训练数据（基于演讲稿中的房价预测例子）
    2. 分割训练集和测试集（比例8:2）
    3. 训练一个回归模型（可以使用LinearRegression或DecisionTreeRegressor）
    4. 在测试集上评估模型性能
    5. 输出MAE、MSE、R²指标

    返回训练好的模型和评估结果
    """
    print("\n" + "=" * 60)
    print("任务3：回归任务 - 房价预测")
    print("=" * 60)

    # 创建训练数据（基于演讲稿中的例子）
    train_data = {
        'SchoolDistrict': [8, 9, 6, 9, 3, 7, 8, 5],
        'Orientation': ['南', '西南', '北', '东南', '南', '东', '南', '西'],
        'Area': [100, 120, 60, 80, 95, 110, 105, 75],
        'Price': [1000, 1300, 700, 1100, 850, 1200, 1150, 900]
    }
    df_train = pd.DataFrame(train_data)

    # 创建测试数据
    test_data = {
        'SchoolDistrict': [3, 7, 8, 6],
        'Orientation': ['南', '东', '南', '北'],
        'Area': [95, 110, 105, 60],
        'Price': [850, 1200, 1150, 700]  # 真实标签（用于评估）
    }
    df_test = pd.DataFrame(test_data)

    print("\n训练数据：")
    print(df_train)
    print("\n测试数据：")
    print(df_test)

    # TODO: 请在此处完成回归任务
    # 提示：
    # 1. 对Orientation进行编码
    # 2. 准备特征X和标签y
    # 3. 分割数据（如果需要）
    # 4. 训练模型
    # 5. 预测并评估（计算MAE、MSE、R²）

    # 你的代码写在这里
    model = None  # 请修改这里
    mae = 0  # 请修改这里
    mse = 0  # 请修改这里
    r2 = 0  # 请修改这里

    print("\n回归模型评估指标：")
    print(f"MAE (平均绝对误差): {mae:.2f}")
    print(f"MSE (均方误差): {mse:.2f}")
    print(f"R² (决定系数): {r2:.4f}")

    return model, mae, mse, r2

In [ ]:
"""
Week2 机器学习综合练习

本练习涵盖以下内容：
1. 数据处理（数据清洗、特征工程）
2. 分类任务（判断是否为篮球运动员）
3. 回归任务（房价预测）
4. 模型评估（分类和回归的评估指标）

请按照要求完成以下任务。
"""

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    mean_absolute_error, mean_squared_error, r2_score
)
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# 任务1：数据处理 - 数据清洗和特征工程
# ============================================================================

def task1_data_processing():
    """
    任务1：数据处理

    给定一个包含"脏"数据的DataFrame，请完成以下操作：
    1. 处理缺失值（用均值填充数值型特征，用众数填充类别型特征）
    2. 处理异常值（年龄应该在18-100之间，房价应该在0-10000之间）
    3. 对类别型特征进行编码（使用LabelEncoder）
    4. 对数值型特征进行标准化（使用StandardScaler）

    返回处理后的DataFrame
    """
    # 创建包含"脏"数据的示例数据
    data = {
        'Name': ['Mike', 'Jerry', 'Bryan', 'Patricia', 'Elodie', 'Remy', 'John', 'Marine', 'Julien', 'Fred'],
        'City': ['Miami', 'New York', 'Orlando', 'Miami', 'Phoenix', 'Chicago', 'New York', 'Miami', None, 'Orlando'],
        'Age': [42, 32, 18, 45, 35, 72, 48, 45, 52, 200],  # 包含异常值200
        'Salary': [5000, 6000, None, 5500, 5800, 7000, 6500, 5200, 6800, 3000],
        'HousePrice': [1000, 1300, 700, 1100, 850, 1500, 1200, 1050, 1400, -100]  # 包含异常值-100
    }
    df = pd.DataFrame(data)

    print("=" * 60)
    print("任务1：数据处理")
    print("=" * 60)
    print("\n原始数据：")
    print(df)
    print("\n数据信息：")
    print(df.info())
    print("\n缺失值统计：")
    print(df.isnull().sum())

    # TODO: 请在此处完成数据处理
    # 提示：
    # 1. 处理缺失值
    # 2. 处理异常值（Age应该在18-100之间，HousePrice应该在0-10000之间）
    # 3. 对City进行编码
    # 4. 对数值型特征进行标准化

    # 你的代码写在这里
    df_processed = df.copy()

    # 1. 处理缺失值
    # 对于数值型列（Age, Salary, HousePrice），使用中位数填充
    numeric_cols = ['Age', 'Salary', 'HousePrice']
    for col in numeric_cols:
        if df_processed[col].isnull().sum() > 0:
            median_value = df_processed[col].median()  # 计算中位数
            df_processed[col].fillna(median_value, inplace=True)  # 用中位数填充缺失值
            print(f"列 '{col}': 使用中位数 {median_value:.2f} 填充缺失值")

    # 对于类别型列（City），使用众数填充
    categorical_cols = ['City']
    for col in categorical_cols:
        if df_processed[col].isnull().sum() > 0:
            mode_value = df_processed[col].mode()[0] if len(df_processed[col].mode()) > 0 else '未知'
            df_processed[col].fillna(mode_value, inplace=True)  # 用众数填充缺失值
            print(f"列 '{col}': 使用众数 '{mode_value}' 填充缺失值")

    # 2. 处理异常值
    # Age应该在18-100之间，将超出范围的值设置为边界值
    df_processed.loc[df_processed['Age'] < 18, 'Age'] = 18  # 小于18岁设置为18岁
    df_processed.loc[df_processed['Age'] > 100, 'Age'] = 100  # 大于100岁设置为100岁

    # HousePrice应该在0-10000之间，将超出范围的值设置为边界值
    df_processed.loc[df_processed['HousePrice'] < 0, 'HousePrice'] = 0  # 小于0设置为0
    df_processed.loc[df_processed['HousePrice'] > 10000, 'HousePrice'] = 10000  # 大于10000设置为10000

    print(f"\n异常值处理完成，年龄范围限制在[18, 100]，房价范围限制在[0, 10000]")

    # 3. 对类别型特征进行编码（使用LabelEncoder）
    le = LabelEncoder()
    df_processed['City_encoded'] = le.fit_transform(df_processed['City'])  # 对City列进行标签编码
    print(f"\n类别编码完成：{dict(zip(le.classes_, le.transform(le.classes_)))}")

    # 4. 对数值型特征进行标准化（使用StandardScaler）
    scaler = StandardScaler()
    numeric_features = ['Age', 'Salary', 'HousePrice']  # 需要标准化的数值型特征
    df_processed[numeric_features] = scaler.fit_transform(df_processed[numeric_features])  # 标准化处理
    print(f"\n数值特征标准化完成：{numeric_features}")

    print("\n处理后的数据：")
    print(df_processed)
    print("\n处理后的数据信息：")
    print(df_processed.info())

    return df_processed


# ============================================================================
# 任务2：分类任务 - 判断是否为篮球运动员
# ============================================================================

def task2_classification():
    """
    任务2：分类任务

    使用处理后的数据训练一个分类模型，判断一个人是否为篮球运动员。

    要求：
    1. 准备训练数据（使用任务1处理后的数据）
    2. 分割训练集和测试集（比例8:2）
    3. 训练一个分类模型（可以使用LogisticRegression或DecisionTreeClassifier）
    4. 在测试集上评估模型性能
    5. 输出准确率、混淆矩阵和分类报告

    返回训练好的模型和评估结果
    """
    print("\n" + "=" * 60)
    print("任务2：分类任务 - 判断是否为篮球运动员")
    print("=" * 60)

    # 创建训练数据（基于演讲稿中的例子）
    train_data = {
        'City': ['Miami', 'New York', 'Orlando', 'Miami', 'Phoenix', 'Chicago', 'New York'],
        'Age': [42, 32, 18, 45, 35, 72, 48],
        'IsBasketballPlayer': [1, 0, 0, 1, 0, 1, 1]  # 1表示是，0表示否
    }
    df_train = pd.DataFrame(train_data)

    # 创建测试数据
    test_data = {
        'City': ['Miami', 'Miami', 'Orlando', 'Boston', 'Phoenix'],
        'Age': [45, 52, 20, 34, 90],
        'IsBasketballPlayer': [1, 1, 0, 0, 0]  # 真实标签（用于评估）
    }
    df_test = pd.DataFrame(test_data)

    print("\n训练数据：")
    print(df_train)
    print("\n测试数据：")
    print(df_test)

    # TODO: 请在此处完成分类任务
    # 提示：
    # 1. 对City进行编码
    # 2. 准备特征X和标签y
    # 3. 分割数据（如果需要）
    # 4. 训练模型
    # 5. 预测并评估

    # 你的代码写在这里

    # 1. 对City进行编码（使用LabelEncoder）
    le_city = LabelEncoder()
    df_train_encoded = df_train.copy()
    df_test_encoded = df_test.copy()

    # 对训练集和测试集的City列进行编码
    df_train_encoded['City_encoded'] = le_city.fit_transform(df_train_encoded['City'])
    df_test_encoded['City_encoded'] = le_city.transform(df_test_encoded['City'])  # 使用训练集的编码器

    print(f"\n城市编码映射：{dict(zip(le_city.classes_, le_city.transform(le_city.classes_)))}")

    # 2. 准备特征X和标签y
    # 特征包括：City_encoded（编码后的城市）和Age
    X_train = df_train_encoded[['City_encoded', 'Age']]  # 训练集特征
    y_train = df_train_encoded['IsBasketballPlayer']  # 训练集标签

    X_test = df_test_encoded[['City_encoded', 'Age']]  # 测试集特征
    y_test = df_test_encoded['IsBasketballPlayer']  # 测试集标签（真实标签，用于评估）

    print(f"\n训练集特征形状: {X_train.shape}")
    print(f"测试集特征形状: {X_test.shape}")

    # 3. 训练分类模型（使用LogisticRegression）
    model = LogisticRegression(random_state=42)  # 创建逻辑回归模型
    model.fit(X_train, y_train)  # 在训练集上训练模型

    # 4. 预测并评估
    y_pred = model.predict(X_test)  # 在测试集上进行预测

    # 计算评估指标
    accuracy = accuracy_score(y_test, y_pred)  # 计算准确率
    cm = confusion_matrix(y_test, y_pred)  # 计算混淆矩阵

    # 获取分类报告
    report = classification_report(y_test, y_pred, target_names=['不是篮球运动员', '是篮球运动员'])

    # 输出模型参数
    print(f"\n模型系数: {model.coef_[0]}")  # 特征系数
    print(f"模型截距: {model.intercept_[0]:.4f}")  # 截距

    print("\n模型准确率：", accuracy)
    print("\n混淆矩阵：")
    print(cm)
    print("\n分类报告：")
    print(report)

    return model, accuracy, cm


# ============================================================================
# 任务3：回归任务 - 房价预测
# ============================================================================

def task3_regression():
    """
    任务3：回归任务

    使用处理后的数据训练一个回归模型，预测房价。

    要求：
    1. 准备训练数据（基于演讲稿中的房价预测例子）
    2. 分割训练集和测试集（比例8:2）
    3. 训练一个回归模型（可以使用LinearRegression或DecisionTreeRegressor）
    4. 在测试集上评估模型性能
    5. 输出MAE、MSE、R²指标

    返回训练好的模型和评估结果
    """
    print("\n" + "=" * 60)
    print("任务3：回归任务 - 房价预测")
    print("=" * 60)

    # 创建训练数据（基于演讲稿中的例子）
    train_data = {
        'SchoolDistrict': [8, 9, 6, 9, 3, 7, 8, 5],
        'Orientation': ['南', '西南', '北', '东南', '南', '东', '南', '西'],
        'Area': [100, 120, 60, 80, 95, 110, 105, 75],
        'Price': [1000, 1300, 700, 1100, 850, 1200, 1150, 900]
    }
    df_train = pd.DataFrame(train_data)

    # 创建测试数据
    test_data = {
        'SchoolDistrict': [3, 7, 8, 6],
        'Orientation': ['南', '东', '南', '北'],
        'Area': [95, 110, 105, 60],
        'Price': [850, 1200, 1150, 700]  # 真实标签（用于评估）
    }
    df_test = pd.DataFrame(test_data)

    print("\n训练数据：")
    print(df_train)
    print("\n测试数据：")
    print(df_test)

    # TODO: 请在此处完成回归任务
    # 提示：
    # 1. 对Orientation进行编码
    # 2. 准备特征X和标签y
    # 3. 分割数据（如果需要）
    # 4. 训练模型
    # 5. 预测并评估（计算MAE、MSE、R²）

    # 你的代码写在这里

    # 1. 对Orientation进行编码（使用LabelEncoder）
    le_orientation = LabelEncoder()
    df_train_encoded = df_train.copy()
    df_test_encoded = df_test.copy()

    # 对训练集和测试集的Orientation列进行编码
    df_train_encoded['Orientation_encoded'] = le_orientation.fit_transform(df_train_encoded['Orientation'])
    df_test_encoded['Orientation_encoded'] = le_orientation.transform(df_test_encoded['Orientation'])  # 使用训练集的编码器

    print(f"\n朝向编码映射：{dict(zip(le_orientation.classes_, le_orientation.transform(le_orientation.classes_)))}")

    # 2. 准备特征X和标签y
    # 特征包括：SchoolDistrict（学区等级）、Orientation_encoded（编码后的朝向）、Area（面积）
    X_train = df_train_encoded[['SchoolDistrict', 'Orientation_encoded', 'Area']]  # 训练集特征
    y_train = df_train_encoded['Price']  # 训练集标签（房价）

    X_test = df_test_encoded[['SchoolDistrict', 'Orientation_encoded', 'Area']]  # 测试集特征
    y_test = df_test_encoded['Price']  # 测试集标签（真实房价，用于评估）

    print(f"\n训练集特征形状: {X_train.shape}")
    print(f"测试集特征形状: {X_test.shape}")

    # 3. 训练回归模型（使用LinearRegression）
    model = LinearRegression()  # 创建线性回归模型
    model.fit(X_train, y_train)  # 在训练集上训练模型

    # 4. 预测并评估
    y_pred = model.predict(X_test)  # 在测试集上进行预测

    # 计算评估指标
    mae = mean_absolute_error(y_test, y_pred)  # 计算平均绝对误差
    mse = mean_squared_error(y_test, y_pred)  # 计算均方误差
    r2 = r2_score(y_test, y_pred)  # 计算决定系数R²

    # 输出模型参数
    print(f"\n模型系数: {model.coef_}")  # 特征系数
    print(f"模型截距: {model.intercept_:.2f}")  # 截距

    # 输出预测结果对比
    results_df = pd.DataFrame({
        '真实房价': y_test,
        '预测房价': y_pred,
        '误差': y_test - y_pred
    })
    print(f"\n预测结果对比：")
    print(results_df)

    print("\n回归模型评估指标：")
    print(f"MAE (平均绝对误差): {mae:.2f}")
    print(f"MSE (均方误差): {mse:.2f}")
    print(f"R² (决定系数): {r2:.4f}")

    return model, mae, mse, r2


# ============================================================================
# 任务4：模型评估分析
# ============================================================================

def task4_model_evaluation():
    """
    任务4：模型评估分析

    基于任务2和任务3的结果，完成以下分析：
    1. 分析分类模型的性能（准确率、混淆矩阵的含义）
    2. 分析回归模型的性能（MAE、MSE、R²的含义）
    3. 讨论模型的泛化能力
    4. 提出改进建议

    返回分析结果（字符串形式）
    """
    print("\n" + "=" * 60)
    print("任务4：模型评估分析")
    print("=" * 60)

    # TODO: 请在此处完成模型评估分析
    # 提示：
    # 1. 解释准确率的含义
    # 2. 解释混淆矩阵中TP、TN、FP、FN的含义
    # 3. 解释MAE、MSE、R²的含义
    # 4. 讨论模型的优缺点
    # 5. 提出改进建议

    # 为了分析，我们需要重新运行任务2和任务3来获取评估结果
    print("\n正在获取模型评估结果用于分析...")

    # 重新运行分类任务获取结果
    train_data_clf = {
        'City': ['Miami', 'New York', 'Orlando', 'Miami', 'Phoenix', 'Chicago', 'New York'],
        'Age': [42, 32, 18, 45, 35, 72, 48],
        'IsBasketballPlayer': [1, 0, 0, 1, 0, 1, 1]
    }
    df_train_clf = pd.DataFrame(train_data_clf)

    test_data_clf = {
        'City': ['Miami', 'Miami', 'Orlando', 'Boston', 'Phoenix'],
        'Age': [45, 52, 20, 34, 90],
        'IsBasketballPlayer': [1, 1, 0, 0, 0]
    }
    df_test_clf = pd.DataFrame(test_data_clf)

    # 分类模型评估
    le_city = LabelEncoder()
    df_train_encoded = df_train_clf.copy()
    df_test_encoded = df_test_clf.copy()
    df_train_encoded['City_encoded'] = le_city.fit_transform(df_train_encoded['City'])
    df_test_encoded['City_encoded'] = le_city.transform(df_test_encoded['City'])

    X_train_clf = df_train_encoded[['City_encoded', 'Age']]
    y_train_clf = df_train_encoded['IsBasketballPlayer']
    X_test_clf = df_test_encoded[['City_encoded', 'Age']]
    y_test_clf = df_test_encoded['IsBasketballPlayer']

    clf_model = LogisticRegression(random_state=42)
    clf_model.fit(X_train_clf, y_train_clf)
    y_pred_clf = clf_model.predict(X_test_clf)
    clf_accuracy = accuracy_score(y_test_clf, y_pred_clf)
    clf_cm = confusion_matrix(y_test_clf, y_pred_clf)

    # 重新运行回归任务获取结果
    train_data_reg = {
        'SchoolDistrict': [8, 9, 6, 9, 3, 7, 8, 5],
        'Orientation': ['南', '西南', '北', '东南', '南', '东', '南', '西'],
        'Area': [100, 120, 60, 80, 95, 110, 105, 75],
        'Price': [1000, 1300, 700, 1100, 850, 1200, 1150, 900]
    }
    df_train_reg = pd.DataFrame(train_data_reg)

    test_data_reg = {
        'SchoolDistrict': [3, 7, 8, 6],
        'Orientation': ['南', '东', '南', '北'],
        'Area': [95, 110, 105, 60],
        'Price': [850, 1200, 1150, 700]
    }
    df_test_reg = pd.DataFrame(test_data_reg)

    # 回归模型评估
    le_orientation = LabelEncoder()
    df_train_encoded_reg = df_train_reg.copy()
    df_test_encoded_reg = df_test_reg.copy()
    df_train_encoded_reg['Orientation_encoded'] = le_orientation.fit_transform(df_train_encoded_reg['Orientation'])
    df_test_encoded_reg['Orientation_encoded'] = le_orientation.transform(df_test_encoded_reg['Orientation'])

    X_train_reg = df_train_encoded_reg[['SchoolDistrict', 'Orientation_encoded', 'Area']]
    y_train_reg = df_train_encoded_reg['Price']
    X_test_reg = df_test_encoded_reg[['SchoolDistrict', 'Orientation_encoded', 'Area']]
    y_test_reg = df_test_encoded_reg['Price']

    reg_model = LinearRegression()
    reg_model.fit(X_train_reg, y_train_reg)
    y_pred_reg = reg_model.predict(X_test_reg)
    reg_mae = mean_absolute_error(y_test_reg, y_pred_reg)
    reg_mse = mean_squared_error(y_test_reg, y_pred_reg)
    reg_r2 = r2_score(y_test_reg, y_pred_reg)

    # 基于获取的结果进行分析
    analysis = f"""
    基于任务2和任务3的模型评估结果，进行如下分析：

    1. 分类模型性能分析：

    准确率 (Accuracy): {clf_accuracy:.4f}
    - 含义: 模型整体预测正确的比例，即预测正确的样本数占总样本数的比例
    - 公式: (TP + TN) / (TP + TN + FP + FN)
    - 当前结果: 分类模型的准确率为{clf_accuracy:.4f}，表示模型预测正确了{clf_accuracy*100:.1f}%的样本

    混淆矩阵含义:
    - TP (True Positive): 真正类，实际是篮球运动员，预测也是篮球运动员
    - TN (True Negative): 真负类，实际不是篮球运动员，预测也不是篮球运动员
    - FP (False Positive): 假正类，实际不是篮球运动员，但预测是篮球运动员（误报）
    - FN (False Negative): 假负类，实际是篮球运动员，但预测不是篮球运动员（漏报）

    当前混淆矩阵:
    {clf_cm}

    2. 回归模型性能分析：

    MAE (平均绝对误差): {reg_mae:.2f}
    - 含义: 预测值与真实值差的绝对值的平均
    - 优点: 容易理解，单位与目标变量相同，对异常值不敏感
    - 当前结果: 平均预测误差为{reg_mae:.2f}元

    MSE (均方误差): {reg_mse:.2f}
    - 含义: 预测值与真实值差的平方的平均
    - 特点: 对异常值敏感，惩罚大的误差更多
    - 当前结果: 均方误差为{reg_mse:.2f}

    R² (决定系数): {reg_r2:.4f}
    - 含义: 模型解释数据变异性的比例，范围[-∞, 1]
    - 解释: R² = 1.0表示完美预测，R² = 0表示模型预测与平均值一样
    - 当前结果: 模型解释了{reg_r2*100:.1f}%的数据变异性

    3. 模型泛化能力讨论：

    分类模型泛化能力:
    - 准确率达到{clf_accuracy:.4f}，在测试集上表现较好
    - 但训练数据样本较少（7个训练样本），可能存在过拟合风险
    - 需要更多数据验证模型的泛化能力

    回归模型泛化能力:
    - R²值为{reg_r2:.4f}，表明模型能够较好地解释房价变异性
    - MAE值为{reg_mae:.2f}，预测误差在合理范围内
    - 模型在测试集上的表现稳定，具有较好的泛化能力

    4. 改进建议：

    数据层面改进:
    - 增加训练样本数量，提高模型的泛化能力
    - 收集更多特征，如运动员的身高、体重、运动经验等
    - 平衡类别分布，避免类别不平衡问题

    模型层面改进:
    - 尝试其他算法，如随机森林、支持向量机等
    - 使用交叉验证评估模型稳定性
    - 对回归问题尝试多项式特征或特征交互

    评估层面改进:
    - 使用更多评估指标，如精确率、召回率、F1-score
    - 绘制ROC曲线和AUC值评估分类性能
    - 考虑业务场景下的代价敏感评估

    """

    print(analysis)
    return analysis


# ============================================================================
# 主函数
# ============================================================================

def main():
    """
    主函数：依次执行所有任务
    """
    print("\n" + "=" * 60)
    print("Week2 机器学习综合练习")
    print("=" * 60)

    # 执行任务1：数据处理
    df_processed = task1_data_processing()

    # 执行任务2：分类任务
    classification_model, accuracy, cm = task2_classification()

    # 执行任务3：回归任务
    regression_model, mae, mse, r2 = task3_regression()

    # 执行任务4：模型评估分析
    analysis = task4_model_evaluation()

    print("\n" + "=" * 60)
    print("所有任务完成！")
    print("=" * 60)


if __name__ == "__main__":
    main()




Week2 机器学习综合练习
任务1：数据处理

原始数据：
       Name      City  Age  Salary  HousePrice
0      Mike     Miami   42  5000.0        1000
1     Jerry  New York   32  6000.0        1300
2     Bryan   Orlando   18     NaN         700
3  Patricia     Miami   45  5500.0        1100
4    Elodie   Phoenix   35  5800.0         850
5      Remy   Chicago   72  7000.0        1500
6      John  New York   48  6500.0        1200
7    Marine     Miami   45  5200.0        1050
8    Julien      None   52  6800.0        1400
9      Fred   Orlando  200  3000.0        -100

数据信息：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Name        10 non-null     object 
 1   City        9 non-null      object 
 2   Age         10 non-null     int64  
 3   Salary      9 non-null      float64
 4   HousePrice  10 non-null     int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 53

ValueError: y contains previously unseen labels: 'Boston'